# Audio Based Gender Classification Model Development

In [4]:
# Importing necessary libraries

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split

In [7]:
# Load the dataset  (The data is already preprocessed audio files converted into .npy)

X = np.load("/content/drive/MyDrive/Colab Notebooks/features.npy")
y = np.load("/content/drive/MyDrive/Colab Notebooks/labels.npy")

In [9]:
# Splitting the data into Train, Test and Validation

# Train Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7)

# Train Validation
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=7)

In [11]:
# Creating the model

model = Sequential()
model.add(Dense(256, input_shape=(128,)))
model.add(Dropout(0.3))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
# one output neuron with sigmoid activation function, 0 means female, 1 means male
model.add(Dense(1, activation="sigmoid"))
# using binary crossentropy as it's male/female classification (binary)
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
# print summary of the model
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 156,545 (611.50 KB)

 Trainable params: 156,545 (611.50 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100
# train the model using the training set and validating using validation set
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[tensorboard, early_stopping])

Epoch 1/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7115 - loss: 0.6693 - val_accuracy: 0.8419 - val_loss: 0.3899
Epoch 2/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8333 - loss: 0.4150 - val_accuracy: 0.8672 - val_loss: 0.3747
Epoch 3/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8481 - loss: 0.3903 - val_accuracy: 0.8736 - val_loss: 0.3207
Epoch 4/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8605 - loss: 0.3642 - val_accuracy: 0.8815 - val_loss: 0.3085
Epoch 5/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8632 - loss: 0.3543 - val_accuracy: 0.8866 - val_loss: 0.2903
Epoch 6/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8677 - loss: 0.3417 - val_accuracy: 0.8880 - val_loss: 0.3040
Epoch 7/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8740 - loss: 0.3224 - val_accuracy: 0.8944 - val_loss: 0.2803
Epoch 8/100
848/848 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.8785 - loss: 0.3209 - val_

In [16]:
# save the model to a file
model.save("results/model.h5")

In [15]:
# Evaluating the model

print(f"Evaluating the model using {len(X_test)} samples...")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Evaluating the model using 6694 samples...
Loss: 0.2200
Accuracy: 91.77%
